# Defining imports:

In [1]:
# Standard library imports
import os
import time
from pathlib import Path

# Third-party library imports
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from torch.nn import TripletMarginLoss
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.models import resnet50
import torchvision.io as io
import torchvision.transforms as transforms
from typing import Tuple, List, Dict, Union, Any

In [2]:
# Matplotlib graph settings
# plt.rcParams["savefig.bbox"] = 'tight'

In [3]:
# Check if CUDA (GPU support) is available and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Using device: {device}")

Using device: cuda


# Loading ground truth:

In [4]:
# Read the CSV file
ground_truth_df = pd.read_csv("ISIC2018_Task3_Test_GroundTruth.csv", dtype={"MEL": int})

# Rename the first column to "Image"
ground_truth_df = ground_truth_df.rename(columns={ground_truth_df.columns[0]: "Image"})

# Add '.jpg' to every value in the 'image' column
ground_truth_df['Image'] = ground_truth_df['Image'] + '.jpg'

# Extract the first two columns and rename the second column
ground_truth_df = ground_truth_df.iloc[:, :2]  # Extracting first two columns
ground_truth_df = ground_truth_df.rename(columns={"MEL": "Ground truth labels"})  # Renaming the second column

In [5]:
# Display the DataFrame
display(ground_truth_df)

,Image,Ground truth labels
0,ISIC_0034524.jpg,0
1,ISIC_0034525.jpg,0
2,ISIC_0034526.jpg,0
3,ISIC_0034527.jpg,0
4,ISIC_0034528.jpg,0
...,...,...
1507,ISIC_0036060.jpg,0
1508,ISIC_0036061.jpg,0
1509,ISIC_0036062.jpg,0
1510,ISIC_0036063.jpg,0


# Model:

**Defining custom model: Triplet Neural Network (with dataset class):**

In [6]:
# Utilize PyTorch's data loading utilities
class TripletDataset(torch.utils.data.Dataset):
    
    # Initialization
    def __init__(self, dataset, transform=None):
        
        # Initializes dataset and transformations
        self.dataset = dataset
        self.transform = transform
        
        # Extract label from dataset
        self.labels = [item[1] for item in dataset.imgs]
        
        # Create dictionary where keys are labels and values are lists of indices corresponding to each label.
        self.label_to_indices = {label: np.where(np.array(self.labels) == label)[0]
                                 for label in set(self.labels)}

        
    # Defines how individual items are retrieved from the dataset given an index (Called when dataset is indexed like dataset[index])
    def __getitem__(self, index):
        
        # Extracts the image path and label of the anchor image at the given index from the dataset.
        # label1 will be label of anchor/positive 
        img1, label1 = self.dataset.imgs[index]
        
        # Initialize positive index with the anchor index
        positive_index = index
        
        # For positive index: randomly selects another index from the indices of images with the same label as the anchor image 
        while positive_index == index:
            positive_index = np.random.choice(self.label_to_indices[label1])
            
        # For negative label: Randomly selects a label that is different from the label of the anchor image 
        negative_label = np.random.choice(list(set(self.labels) - set([label1])))
        negative_index = np.random.choice(self.label_to_indices[negative_label])
        
        # Load images corresponding to the anchor, positive, and negative indices and convert images to RGB format
        img2 = self.dataset.imgs[positive_index][0]
        img3 = self.dataset.imgs[negative_index][0]
        img1 = Image.open(img1).convert("RGB")
        img2 = Image.open(img2).convert("RGB")
        img3 = Image.open(img3).convert("RGB")
        
        # If transformation is not None, apply the transformation
        if self.transform is not None:
            img1 = self.transform(img1)
            img2 = self.transform(img2)
            img3 = self.transform(img3)
        
        # Return images
        return label1, img1, img2, img3

    # Return the length of the dataset
    def __len__(self):
        return len(self.dataset)

In [7]:
class Identity(nn.Module):
    def forward(self, x):
        return x

In [8]:
class TripletNetwork(nn.Module):
    def __init__(self, embedding_size=64):
        super(TripletNetwork, self).__init__()
        
        # Load a pre-trained resnet50 model
        self.backbone = resnet50(pretrained=True)
        
        # Replace the fully connected layer with an Identity module
        self.backbone.fc = Identity()
        
        # Embedding layer
        self.embedding_layer = nn.Linear(2048, embedding_size)  # Use 2048 as the in_features to match ResNet-50
        
        # Classification layers
        self.fc = nn.Sequential(nn.ReLU(), nn.Dropout(0.7), nn.Linear(embedding_size, 1), nn.Sigmoid())

    def forward(self, x, return_embedding=False):
        # Extract features using the backbone
        x = self.backbone(x)  # This will now give a [batch_size, 2048] tensor directly
        
        # Get the embedding
        embedding = self.embedding_layer(x)
        
        if return_embedding:
            return embedding
        
        # Pass embedding through the classification layer
        x = self.fc(embedding)
        return x

# Loading images into Data Loaders:

**Loading images into test data loader:**

In [9]:
class CustomImageFolder(ImageFolder):
    @staticmethod
    def find_classes(directory: Union[str, Path]) -> Tuple[List[str], Dict[str, int]]:
        """
        Finds the class folders in a dataset structured in a directory by overriding the sorting order.

        Parameters:
            directory (Union[str, Path]): Root directory path.

        Returns:
            Tuple[List[str], Dict[str, int]]: (classes, class_to_idx) where classes are a list of 
                                              the class names and class_to_idx is a dictionary mapping 
                                              class name to class index.
        """
        # Correct the syntax error by adding an extra set of parentheses around the generator expression
        classes = sorted((entry.name for entry in os.scandir(directory) if entry.is_dir()), reverse=True)
        if not classes:
            raise FileNotFoundError(f"Couldn't find any class folder in {directory}.")
        class_to_idx = {cls_name: i for i, cls_name in enumerate(classes)}
        return classes, class_to_idx

In [10]:
# Set train and valid directory paths
test_directory = 'data/Test'

# Batch size
bs = 32

#define a standard transform to tensor
transform = transforms.Compose([transforms.ToTensor()])

# Define the index mapping for folders to labels
folder_to_label = {'no_melanoma': 0, 'melanoma': 1}

# Load Data from folders
data = {
    'test': CustomImageFolder(root=test_directory, transform=transform)
}

In [11]:
print(data['test'].class_to_idx)

{'no_melanoma': 0, 'melanoma': 1, 'Test_All': 2}


In [12]:
test_dataset = TripletDataset(dataset=data['test'], transform=transform)
test_data = DataLoader(test_dataset, batch_size=bs, shuffle=False)

# Predict One Image:

In [ ]:
# Initiliaze the Triplet Network 
loaded_model = TripletNetwork(embedding_size=64) 

# Load the model from model_path
loaded_model.load_state_dict(torch.load("model_weights.pth"))

# Set it in eval mode
#loaded_model.eval()

# Move model to device
loaded_model = loaded_model.to(device)  

In [ ]:
def predict_class(model, image, reference_embeddings, device):
    
    # Switch model to evaluation mode
    model.eval()  
    
    
    with torch.no_grad():
        # Make sure the image has a batch dimension
        if image.dim() == 3:
            image = image.unsqueeze(0)  # Add batch dimension if not present
        image = image.to(device)
        
        print("Input shape to model:", image.shape)
        
        # Get the embedding of the uploaded image
        image_embedding = model(image)

    # Initialize the closest class and smallest distance
    closest_class = None
    smallest_distance = float('inf')

    # Compare the uploaded image's embedding to each reference embedding
    for class_name, ref_embedding in reference_embeddings.items():
        distance = (image_embedding - ref_embedding.to(device)).pow(2).sum(1).item()
        if distance < smallest_distance:
            smallest_distance = distance
            closest_class = class_name

    return closest_class

In [ ]:
def load_image(image_path, transform, device):
    image = Image.open(image_path).convert('RGB')
    image = transform(image)
    image = image.unsqueeze(0).to(device)  # Add batch dimension and send to device
    return image

# Define the transform
transform = transforms.Compose([
    transforms.ToTensor(),
])

# Example image path
image_path = "data/Test/melanoma/ISIC_0035914_v1.jpg"

# Extract image name only
image_file_name = os.path.basename(image_path)
#print(image_file_name)

# Load the image
image = load_image(image_path, transform, device)

# Load embeddings
reference_embeddings = torch.load('reference_embeddings.pt')

# Now you can directly use the loaded image for prediction
predicted_class = predict_class(loaded_model, image, reference_embeddings, device)

In [ ]:
# Display the DataFrame
display(ground_truth_df)

In [ ]:
# Function to get ground truth label for a given image name
def get_ground_truth(image_name):
    
    # Check if the image name exists in the DataFrame
    if image_name in ground_truth_df['Image'].values:
        
        # Filter the DataFrame for the given image name and get the corresponding ground truth label
        ground_truth_label = ground_truth_df.loc[ground_truth_df['Image'] == image_name, 'Ground truth labels'].iloc[0]
        return ground_truth_label
    
    else:
        raise ValueError("Image not found in DataFrame")

In [ ]:
converted_image_file_name = image_file_name.replace('_v1', '')
#print(converted_image_file_name)

actual_class = get_ground_truth(converted_image_file_name)

In [ ]:
if predicted_class == 1:
    print("Image is predicted to contain Melanoma, Class 1")
else:
    print("Image is predicted to not contain No Melanoma, Class 0")

In [ ]:
if actual_class == 1:
    print("Ground-truth value for the Image is Melanoma, Class 1")
else:
    print("Ground-truth value for the Image is No Melanoma, Class 0")

# Predict Multiple Images:

In [13]:
class CustomImageFolderPrediction(ImageFolder):
    @staticmethod
    def find_classes(directory: Union[str, os.PathLike]) -> Tuple[List[str], Dict[str, int]]:
        """
        Finds the class folders in a dataset structured in a directory by overriding the sorting order.

        Parameters:
            directory (Union[str, os.PathLike]): Root directory path.

        Returns:
            Tuple[List[str], Dict[str, int]]: (classes, class_to_idx) where classes are a list of 
                                              the class names sorted in reverse alphabetical order, 
                                              and class_to_idx is a dictionary mapping class name to class index.
        """
        classes = sorted([entry.name for entry in os.scandir(directory) if entry.is_dir()], reverse=True)
        class_to_idx = {cls_name: i for i, cls_name in enumerate(classes)}
        return classes, class_to_idx

    def __getitem__(self, index: int):
        """
        Override the __getitem__ method to return the path along with the image and label.

        Parameters:
            index (int): Index of the item.

        Returns:
            tuple: (image, label, path) where image is the transformed image tensor, label is the class label of the image, 
                   and path is the file path of the image.
        """
        # Call the original __getitem__ to get the image and label
        original_tuple = super(CustomImageFolderPrediction, self).__getitem__(index)
        # Get the image path
        path = self.imgs[index][0]  # self.imgs is a list of (image path, class index) tuples
        # Return a new tuple that includes the original content plus the path
        return original_tuple + (path,)

In [16]:
# Initiliaze the Triplet Network 
loaded_model = TripletNetwork(embedding_size=64) 

# Load the model from model_path
#loaded_model.load_state_dict(torch.load("model_weights_3_epoch_Triplet.pth"))
loaded_model.load_state_dict(torch.load("saved_files/model_weights.pth"))

# Set it in eval mode
#loaded_model.eval()

# Move model to device
loaded_model = loaded_model.to(device)  

In [17]:
test_dataset_predict = CustomImageFolderPrediction(root = test_directory, transform=transform)
test_dataloader_predict = DataLoader(test_dataset_predict, batch_size=bs, shuffle=False)

# Load embeddings
reference_embeddings = torch.load('saved_files/reference_embeddings.pt')

In [18]:
def predict_classes(model, data_loader, reference_embeddings, device):
    model.eval() 
    predictions = {}

    with torch.no_grad():
        for images, _, paths in data_loader:  # Assuming paths are returned by the DataLoader
            images = images.to(device)
            image_embeddings = model(images)

            for i in range(images.size(0)):
                smallest_distance = float('inf')
                closest_class = None
                
                for class_name, ref_embedding in reference_embeddings.items():
                    distance = (image_embeddings[i] - ref_embedding.to(device)).pow(2).sum().item()
                    if distance < smallest_distance:
                        smallest_distance = distance
                        closest_class = class_name
                
                filename = os.path.basename(paths[i])
                predictions[filename] = closest_class

    return predictions

In [19]:
predicted_classes = predict_classes(loaded_model, test_dataloader_predict, reference_embeddings, device)

In [20]:
print(predicted_classes)

{'0_ISIC_0034524_v1.jpg': 0, '0_ISIC_0034525_v1.jpg': 0, '0_ISIC_0034526_v1.jpg': 0, '0_ISIC_0034527_v1.jpg': 1, '0_ISIC_0034528_v1.jpg': 0, '0_ISIC_0034530_v1.jpg': 0, '0_ISIC_0034531_v1.jpg': 1, '0_ISIC_0034532_v1.jpg': 0, '0_ISIC_0034533_v1.jpg': 0, '0_ISIC_0034534_v1.jpg': 0, '0_ISIC_0034535_v1.jpg': 1, '0_ISIC_0034536_v1.jpg': 1, '0_ISIC_0034537_v1.jpg': 0, '0_ISIC_0034538_v1.jpg': 1, '0_ISIC_0034539_v1.jpg': 1, '0_ISIC_0034540_v1.jpg': 0, '0_ISIC_0034541_v1.jpg': 0, '0_ISIC_0034542_v1.jpg': 0, '0_ISIC_0034543_v1.jpg': 0, '0_ISIC_0034544_v1.jpg': 0, '0_ISIC_0034545_v1.jpg': 0, '0_ISIC_0034546_v1.jpg': 0, '0_ISIC_0034547_v1.jpg': 1, '0_ISIC_0034549_v1.jpg': 0, '0_ISIC_0034551_v1.jpg': 0, '0_ISIC_0034552_v1.jpg': 1, '0_ISIC_0034553_v1.jpg': 0, '0_ISIC_0034554_v1.jpg': 0, '0_ISIC_0034555_v1.jpg': 0, '0_ISIC_0034556_v1.jpg': 0, '0_ISIC_0034557_v1.jpg': 0, '0_ISIC_0034558_v1.jpg': 0, '0_ISIC_0034559_v1.jpg': 0, '0_ISIC_0034560_v1.jpg': 0, '0_ISIC_0034561_v1.jpg': 0, '0_ISIC_0034562_v1.

In [21]:
# Predicted class dataframe
predicted_classes_df = pd.DataFrame(list(predicted_classes.items()), columns=['Image', 'Predicted Value'])

# Remove '_v1' from the 'Filename' column
predicted_classes_df['Image'] = predicted_classes_df['Image'].str.replace('_v1', '')

# Display the DataFrame
display(predicted_classes_df)

,Image,Predicted Value
0,0_ISIC_0034524.jpg,0
1,0_ISIC_0034525.jpg,0
2,0_ISIC_0034526.jpg,0
3,0_ISIC_0034527.jpg,1
4,0_ISIC_0034528.jpg,0
...,...,...
3019,ISIC_0036060.jpg,0
3020,ISIC_0036061.jpg,0
3021,ISIC_0036062.jpg,0
3022,ISIC_0036063.jpg,1


**Merged dataframe:**

In [22]:
# Perform inner join on "Image" column
merged_df = pd.merge(predicted_classes_df, ground_truth_df, on="Image")

# Display the merged DataFrame
display(merged_df)

,Image,Predicted Value,Ground truth labels
0,ISIC_0034529.jpg,1,1
1,ISIC_0034548.jpg,1,1
2,ISIC_0034572.jpg,1,1
3,ISIC_0034573.jpg,1,1
4,ISIC_0034584.jpg,0,1
...,...,...,...
1507,ISIC_0036060.jpg,0,0
1508,ISIC_0036061.jpg,0,0
1509,ISIC_0036062.jpg,0,0
1510,ISIC_0036063.jpg,1,0


In [23]:
# Find rows where the Predicted Value is different from the Ground truth labels
different_rows = merged_df[merged_df['Predicted Value'] != merged_df['Ground truth labels']]

# Find rows where the Predicted Value is the same as the Ground truth labels
same_rows = merged_df[merged_df['Predicted Value'] == merged_df['Ground truth labels']]

In [24]:
# Display the different rows
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(different_rows)

,Image,Predicted Value,Ground truth labels
4,ISIC_0034584.jpg,0,1
6,ISIC_0034605.jpg,0,1
8,ISIC_0034630.jpg,0,1
21,ISIC_0034784.jpg,0,1
23,ISIC_0034817.jpg,0,1
28,ISIC_0034873.jpg,0,1
30,ISIC_0034875.jpg,0,1
35,ISIC_0034926.jpg,0,1
38,ISIC_0034957.jpg,0,1
42,ISIC_0034988.jpg,0,1


In [25]:
# Display the different rows
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(same_rows)

,Image,Predicted Value,Ground truth labels
0,ISIC_0034529.jpg,1,1
1,ISIC_0034548.jpg,1,1
2,ISIC_0034572.jpg,1,1
3,ISIC_0034573.jpg,1,1
5,ISIC_0034595.jpg,1,1
7,ISIC_0034628.jpg,1,1
9,ISIC_0034633.jpg,1,1
10,ISIC_0034638.jpg,1,1
11,ISIC_0034644.jpg,1,1
12,ISIC_0034650.jpg,1,1


In [30]:
count = ((different_rows['Predicted Value'] == 0) & (different_rows['Ground truth labels'] == 1)).sum()
print(count)

45


In [31]:
count = ((different_rows['Predicted Value'] == 1) & (different_rows['Ground truth labels'] == 0)).sum()
print(count)

308


In [28]:
print(f"The number of images uploaded were: {len(merged_df)}")
print(f"The number of images classified correctly were {len(same_rows)}")
print(f"The number of images classified incorrectly were {len(different_rows)}")

The number of images uploaded were: 1512
The number of images classified correctly were 1159
The number of images classified incorrectly were 353
